### Text Generation using RNN

##### Importing Libraries

In [1]:
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np 

##### Loading Data

In [4]:
data_clean_path = 'data_clean.pkl'

import pandas as pd
data_clean = pd.read_pickle(data_clean_path)
data_clean.head()

,content,date,retweets,favorites
0,be sure to tune in and watch trump on late nig...,2009-05-04 13:54:25,510,917
1,trump will be on the view tomorrow morning to ...,2009-05-04 20:00:10,34,267
2,trump top ten financial on late show with very...,2009-05-08 08:38:08,13,19
3,new post celebrity apprentice finale and learn...,2009-05-08 15:40:15,11,26
4,my persona will never be that of a wallflower ...,2009-05-12 09:07:28,1375,1945


##### Seperating 2020 tweets

In [5]:
data = ''

for tweet in data_clean.content[40939:]:
    data += '\n' + tweet

len(data)

253074

##### Tokenizing data

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

corpus = data.lower().split("\n")

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

print(total_words) # no of unique words
print(tokenizer.word_index)

3356
{'the': 1, 'to': 2, 'and': 3, 'of': 4, 'a': 5, 'is': 6, 'in': 7, '…': 8, 'for': 9, '’': 10, 'i': 11, 'great': 12, 'will': 13, 'you': 14, 'that': 15, 'our': 16, 'are': 17, 'on': 18, 'be': 19, 'it': 20, 'we': 21, 'they': 22, '“': 23, '”': 24, 'he': 25, 'with': 26, 'have': 27, 'this': 28, 'thank': 29, 't': 30, 'was': 31, 's': 32, 'all': 33, 'very': 34, 'people': 35, 'my': 36, 'by': 37, 'not': 38, 'news': 39, 'at': 40, 'as': 41, 'just': 42, 'fake': 43, 'but': 44, 'do': 45, 'now': 46, 'so': 47, 'who': 48, 'more': 49, 'their': 50, 'big': 51, 'up': 52, 'from': 53, 'new': 54, 'than': 55, 'his': 56, 'no': 57, 'out': 58, 'president': 59, 'never': 60, 'your': 61, 'what': 62, 'get': 63, 'country': 64, 'or': 65, 'job': 66, 'should': 67, 'total': 68, 'us': 69, 'many': 70, 'doing': 71, 'been': 72, 'would': 73, 'military': 74, 'trump': 75, 'nothing': 76, 'complete': 77, 'them': 78, 'even': 79, 'endorsement': 80, 'like': 81, 'house': 82, 'strong': 83, 'state': 84, 'there': 85, 'me': 86, 'done': 87

##### Create predictors and labels 

In [7]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

##### Building and Training the RNN model

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 64, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(20)))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(xs, ys, epochs=500, verbose=1)

Epoch 1/500
1493/1493 [==============================] - 78s 52ms/step - loss: 6.2098 - accuracy: 0.0536
Epoch 2/500
1493/1493 [==============================] - 78s 52ms/step - loss: 5.8545 - accuracy: 0.0731
Epoch 3/500
1493/1493 [==============================] - 79s 53ms/step - loss: 5.6204 - accuracy: 0.1015
Epoch 4/500
1493/1493 [==============================] - 80s 53ms/step - loss: 5.4383 - accuracy: 0.1253
Epoch 5/500
1493/1493 [==============================] - 80s 53ms/step - loss: 5.2899 - accuracy: 0.1405
Epoch 6/500
1493/1493 [==============================] - 80s 53ms/step - loss: 5.1657 - accuracy: 0.1531
Epoch 7/500
1493/1493 [==============================] - 80s 53ms/step - loss: 5.0598 - accuracy: 0.1639
Epoch 8/500
1493/1493 [==============================] - 80s 54ms/step - loss: 4.9651 - accuracy: 0.1733
Epoch 9/500
1493/1493 [==============================] - 79s 53ms/step - loss: 4.8769 - accuracy: 0.1816
Epoch 10/500
1493/1493 [==============================]

##### Save the model locally

In [ ]:
from keras.models import load_model

model.save('model/tweet_generation.h5')

##### Load the saved model

In [10]:
model_trump_tweet_generaation = tf.keras.models.load_model('model/tweet_generation.h5')

### Results

* From the model trained above, it is clear that the accuracy is 51.47%
* Accuracy is quite low and the tweet generated by this model might not be structurally or grammatically correct. 

##### Predicting tweet with some input

In [12]:
def predict_tweet(seed_text, next_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model_trump_tweet_generaation.predict_classes(token_list, verbose=0)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [46]:
print("Original Tweet:\n" + data_clean.content[28054])
print("\nGenerated Tweet:\n" + predict_tweet(" ".join(data_clean.content[28054].split()[:5]), 9))

Original Tweet:
lightweight marco was working hard last night the problem is he is a choker and once a choker always a choker

Generated Tweet:
lightweight marco was working hard for our country a great guy and high crime


In [47]:
print("Original Tweet:\n" + data_clean.content[12608])
print("\nGenerated Tweet:\n" + predict_tweet(" ".join(data_clean.content[12608].split()[:5]), 9))

Original Tweet:
watching the roast of classic

Generated Tweet:
watching the roast of classic after is now done to take away … lacy


In [52]:
print("Original Tweet:\n" + data_clean.content[35543])
print("\nGenerated Tweet:\n" + predict_tweet(" ".join(data_clean.content[35543].split()[:5]), 13))

Original Tweet:
i have our great republican senator from montana to attend his daughter ’ s wedding rather than coming to today ’ s vote was ready to do whatever he had to but we had the necessary number to the family a wonderful day

Generated Tweet:
i have our great republican party in the in the history of the people of the federal reserve


In [54]:
# 20270
print("Original Tweet:\n" + data_clean.content[20270])
print("\nGenerated Tweet:\n" + predict_tweet(" ".join(data_clean.content[20270].split()[:5]), 14))

Original Tweet:
my daughter n i watching very inspiration u r a very humble gentleman love the

Generated Tweet:
my daughter n i watching a great guy and great support of those hall in the universe by far


In [42]:
# 12048
print("Original Tweet:\n" + data_clean.content[12048])
print("\nGenerated Tweet:\n" + predict_tweet(" ".join(data_clean.content[12048].split()[:5]), 14))

Original Tweet:
veteran military dogs campaign to rehabilitate veteran and find them a home

Generated Tweet:
veteran military dogs campaign to greatness reopen chain administration will receive over the world to the party of military
